<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/WMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from wmd import WMD
import wordfreq


In [2]:
!pip install wmd

     |████████████████████████████████| 112kB 7.7MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=631139 sha256=21a64d617f13eaecff562a4ead7429723d3f41f296b7448510a41e56e838db68
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [3]:
!pip install wordfreq
!git clone https://github.com/brmson/dataset-sts
!wget  http://nlp.stanford.edu/data/glove.840B.300d.zip

     |████████████████████████████████| 32.8MB 125kB/s 
     |████████████████████████████████| 4.9MB 47.6MB/s 
     |████████████████████████████████| 276kB 45.7MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=04508ac4b1794e260214d1d419ed2863706d6300d49d66b136ab5468d233e8ed
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=170601a0111a60fae128ebcef7f0a6deaff497929ebf13ba46fbbbfd31d56182
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861360 sha256=cbcf4cbf7c240453112dee789310d89037584d69bf6cad297ffffac6efb93124
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [0]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [2]:
!pwd

/content


In [0]:
glove_pd = pd.read_csv(z.open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
#glove_pd.to_csv('/content/drive/My Drive/NLP_Project/glove_pd.csv')
np.savetxt(r'/content/drive/My Drive/NLP_Project/glove.txt', glove_pd.values, fmt='%d', delimiter=' ')

In [3]:
paraphrase = pd.read_csv('Paraphrase.csv')
paraphrase.describe()

,label
count,998.000000
mean,3.594866
std,0.794834
min,1.000000
25%,3.000000
50%,3.667000
75%,4.333000
max,5.000000


In [0]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(paraphrase.iloc[9,0])
paraphrase['text_1_tokens'] =paraphrase.text_1.apply(lambda x:tokenizer.tokenize(x)) 
paraphrase['text_2_tokens'] =paraphrase.text_2.apply(lambda x:tokenizer.tokenize(x)) 

def embed (sent):
  toklist = tokenizer.tokenize(sent)
  embs = []
  freqs = []
  for word in toklist:
    try: 
      embs.append(glove[word.lower()])
      freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))
    except:
      try:
        embs.append(glove[word.strip("'")])
        freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))

      except:
        continue
  return embs, freqs

embed(paraphrase.iloc[9,0])
text_1_embeddings = paraphrase.text_1.apply(lambda x: embed(x) )
text_2_embeddings = paraphrase.text_2.apply(lambda x: embed(x) )

In [44]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/content/glove.840B.300d.txt')
tmp_file = get_tmpfile('/content/drive/My Drive/NLP_Project/W2V.txt')

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
paraphrase['WMD'] = 0
for i in range(len(paraphrase)):
 paraphrase.iloc[i,-1] =  model.wmdistance(paraphrase.iloc[i,0], paraphrase.iloc[i,1])

In [0]:
#paraphrase = paraphrase.drop(['text_1_tokens','text_2_tokens'],axis = 1)
paraphrase.to_csv('/content/drive/My Drive/NLP_Project/paraphrase.csv')